In [1]:
import common
reload(common)

<module 'common' from 'common.pyc'>

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

class TfIdfFeatureExtractor(object):
    def __init__(self, ngram):
        self.vectorizer = TfidfVectorizer(min_df=0.0005, ngram_range=(1,ngram))
        
    def train(self, train_data):
        features = self.vectorizer.fit_transform(train_data["content"])
        print "Vocab size %d" % len(self.vectorizer.vocabulary_)
        return features
        
    def transform(self, test_data):
        return self.vectorizer.transform(test_data["content"])

In [3]:
common.extract_features(TfIdfFeatureExtractor(1), "vec-tfidf-1")

Loading ka-comments-balanced dataset.
Training feature extractor vec-tfidf-1.
Vocab size 3635
Done.


In [4]:
common.test_features("vec-tfidf-1")

Loading features vec-tfidf-1.
Training models.
##        MultinomialNB     vec-tfidf-1 accuracy: 53.6 %


KeyboardInterrupt: 

In [26]:
common.extract_features(TfIdfFeatureExtractor(2), "vec-tfidf-2")

Training feature extractor vec-tfidf-2.


KeyboardInterrupt: 

In [ ]:
common.test_features("vec-tfidf-2")